In [42]:
!echo $http_proxy

http://squid-compute101:3128


In [43]:
import comet_ml
comet_ml.login()

In [44]:
from comet_ml import Experiment

In [45]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing import image
from keras.optimizers import RMSprop, Adam,SGD


from keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from keras.callbacks import Callback
import logging
import tensorflow as tf

In [46]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255


In [47]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [48]:
def build_vanish_grad(input_shape=(28, 28, 1), numnodes=128):
    model = Sequential()

    # Input layer (Flatten the 28x28 image to 784)
    model.add(Flatten(input_shape=input_shape))  # Flatten layer to reshape (28,28,1) to (784,)

    # First hidden layer
    model.add(Dense(numnodes, activation='sigmoid'))

    # Hidden layers: Increasing depth to induce vanishing gradients
    for _ in range(10):  # Add 10 hidden layers
        model.add(Dense(numnodes, activation='sigmoid'))

    # Output layer
    model.add(Dense(10, activation='softmax'))  # Output for 10-class classification

    return model

# Ensure the correct input shape (28, 28, 1) for the image
model = build_vanish_grad(numnodes=128, input_shape=(28, 28, 1))
# Compile with high learning rate and no gradient control
model.compile(optimizer=SGD(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 266,890 (1.02 MB)

 Trainable params: 266,890 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
experiment = Experiment(project_name="Vanishing_grad")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nast69/vanishing-grad/78185b09b32a4160bcb5714e3bba3dc0

COMET INFO: Couldn't find a Git repository in '/lustre06/project/6078700/nast/comet' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [50]:
experiment.display()

In [51]:
class LoggingCallback(Callback):
    def __init__(self, experiment):
        self.experiment = experiment

    def on_epoch_end(self, epoch, logs=None):
        count = 0
        for weight in self.model.get_weights():
            name = "layer-" + str((count // 2) + 1) + "-" + ("bias" if count % 2 == 1 else "weight")
            self.experiment.log_histogram_3d(weight, name=name)
            count += 1
        

In [2]:
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # Adjust as per your task

# Training loop

for epoch in range(30):
    with tf.GradientTape() as tape:
        predictions = model(x_test)
        loss = loss_fn(y_test, predictions)
    grads = tape.gradient(loss, model.trainable_variables)

            # Log gradients as histograms
    for i, grad in enumerate(grads):
        experiment.log_histogram_3d(
            grad.numpy().flatten(), name=f"Gradient_Layer_{i}", step=epoch
        )

NameError: name 'tf' is not defined

In [53]:
experiment.log_dataset_hash(x_train)

cb = LoggingCallback(experiment)
with experiment.train():
    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=30,
        callbacks=[cb],
        validation_data=(x_test, y_test)
    )

# callbacks=[cb],
score = model.evaluate(x_test, y_test, verbose=0)
logging.info("Score %s", score)
experiment.end()

Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.0927 - loss: 2.5794 - val_accuracy: 0.0892 - val_loss: 2.5469
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0909 - loss: 2.5349 - val_accuracy: 0.0892 - val_loss: 2.5070
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0886 - loss: 2.5008 - val_accuracy: 0.0892 - val_loss: 2.4761
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0904 - loss: 2.4668 - val_accuracy: 0.0892 - val_loss: 2.4516
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0905 - loss: 2.4432 - val_accuracy: 0.0892 - val_loss: 2.4316
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0894 - loss: 2.4262 - val_accuracy: 0.0892 - val_loss: 2.4151
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0871 - loss: 2.4127 - val_accuracy: 0.0892 - val_loss: 2.4013
Epoch 8/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0920 - loss: 2.3971 - 

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : fixed_asphalt_3396
COMET INFO:     url                   : https://www.comet.com/nast69/vanishing-grad/78185b09b32a4160bcb5714e3bba3dc0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_accuracy [30]                 : (0.09035000205039978, 0.11236666887998581)
COMET INFO:     train_batch_accuracy [5640]         : (0.0, 0.1875)
COMET INFO:     train_batch_loss [5640]             : (2.249779462814331, 2.5954062938690186)
COMET INFO:     train_epoch_duration [30]           : (2.287792218005052, 7.385036367995781)
COMET INFO:     train_loss [30]                     : (2.308588981628418, 2.5699682235717773)
COMET INFO:     train_val_accuracy